**https://www.youtube.com/watch?v=vF_qrJ0qHkg**

**https://colab.research.google.com/drive/1wfBW4ekWIkCpmL411x2mTg-UzE3nGJE_?usp=sharing#scrollTo=pPBMr4MuUgqs**

**Mount Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [2]:
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest

**Load Datasets**

In [3]:
Train_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Train.csv')[0:1000]
Test_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Test.csv')[0:1000]
Val_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Val.csv')[0:1000]

In [4]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [5]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [6]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [7]:
Train_data.shape, Val_data.shape, Test_data.shape

((1000, 3), (1000, 3), (1000, 2))

**Text Preprocessing**

In [8]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
Train_data['cleanText'] = Train_data['tweet'].map(lambda s:preprocess(s))
Val_data['cleanText'] = Val_data['tweet'].map(lambda s:preprocess(s))
Test_data['cleanText'] = Test_data['tweet'].map(lambda s:preprocess(s))

In [10]:
Train_data.head()

,id,tweet,label,cleanText
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...


In [11]:
Val_data.head()

,id,tweet,label,cleanText
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...


In [12]:
Test_data.head()

,id,tweet,cleanText
0,1,Our daily update is published. States reported...,daily update published states reported tests n...
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...


In [13]:
Train_data.shape, Val_data.shape, Test_data.shape

((1000, 4), (1000, 4), (1000, 3))

In [14]:
sentences = []

for i in Train_data['cleanText']:
    sentences.append(i)

for j in Val_data['cleanText']:
    sentences.append(i)

for k in Test_data['cleanText']:
    sentences.append(i)

#**Word2Vec Embeddings**

**Install Spacy**

In [15]:
!pip install spacy

In [16]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=7be23247ad89b26c6689d3caba5e1641399182d356862ebf1fb0d9a244c221b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-zveqyxao/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [17]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [18]:
x = 'king man woman'
doc = nlp(x)

for token1 in doc:
    for token2 in doc:
        print((token1.text, token2.text, token1.similarity(token2)))

('king', 'king', 1.0)
('king', 'man', 0.4088461)
('king', 'woman', 0.26556593)
('man', 'king', 0.4088461)
('man', 'man', 1.0)
('man', 'woman', 0.7401745)
('woman', 'king', 0.26556593)
('woman', 'man', 0.7401745)
('woman', 'woman', 1.0)


**Model Building**

In [19]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-_a5nna3u
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-_a5nna3u
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11754 sha256=cd71f6aa7e15f66aa1deb39137fb4759c8529f5e5f4b4645b90687303afe82ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-qtfvwstt/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [20]:
import preprocess_kgptalkie as ps

In [21]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [22]:
x = 'you have @text dAta @#this youuuuuuu'
get_clean(x)

'you have text data this you'

**Word2Vec Conversion**

In [23]:
x = 'cat dog'
doc = nlp(x)

In [24]:
doc.vector.shape

(300,)

In [25]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [26]:
get_vec(x)

array([-2.76215017e-01,  1.73051000e-01, -1.06199495e-01, -2.87514985e-01,
       -6.71409965e-02,  3.10755014e-01, -1.97300002e-01, -2.83794999e-01,
        9.76060033e-02,  1.66744995e+00, -6.34269953e-01,  9.63025019e-02,
       -6.48334980e-01,  1.21810503e-01, -2.99584985e-01,  2.50550002e-01,
        2.70019501e-01,  1.12965000e+00, -1.50799975e-02, -3.55899990e-01,
       -3.66439998e-01, -6.04224950e-02,  5.51749915e-02, -3.11755002e-01,
        3.66990000e-01, -1.74511507e-01, -3.25464994e-01, -4.03584987e-01,
        2.28844002e-01, -1.20778494e-01, -6.23224974e-01,  6.32480010e-02,
       -1.10113002e-01,  4.38295007e-01,  3.05490017e-01,  6.75000250e-03,
        5.48135042e-01, -8.55714977e-01, -1.13398999e-01,  1.28289998e-01,
        2.71515012e-01,  1.46710008e-01, -1.70319498e-01, -3.54690015e-01,
        2.79549956e-02, -1.16696998e-01, -4.24840003e-01,  2.22696498e-01,
       -4.93499637e-03, -2.17053995e-01,  1.82810009e-01,  9.06850100e-02,
        1.11986503e-01,  

In [44]:
get_vec(x).shape

(300,)

**Getting word vectors for text**

In [27]:
#Train data
Train_data['vec'] = Train_data['cleanText'].apply(lambda x: get_vec(x))

#Val data
Val_data['vec'] = Val_data['cleanText'].apply(lambda x: get_vec(x))

#Test data
Test_data['vec'] = Test_data['cleanText'].apply(lambda x: get_vec(x))

In [28]:
Train_data.head(10)

,id,tweet,label,cleanText,vec
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...,"[-0.21722597, 0.22827053, -0.035425685, -0.026..."
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...,"[-0.2625562, 0.2710137, 0.039430913, -0.042415..."
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...,"[-0.20716159, 0.081538565, -0.09774068, 0.0231..."
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...,"[-0.107934915, 0.03293967, 0.15700977, -0.0880..."
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...,"[-0.20053053, 0.22021952, -0.0500075, -0.19174..."
5,6,"Covid Act Now found ""on average each person in...",real,covid act found average person illinois covid ...,"[-0.22577249, 0.27925843, -0.20464352, -0.0794..."
6,7,If you tested positive for #COVID19 and have n...,real,tested positive covid symptoms stay home away ...,"[-0.07399046, 0.24846406, -0.2460248, -0.13207..."
7,8,Obama Calls Trump’s Coronavirus Response A Cha...,fake,obama calls trump coronavirus response chaotic...,"[-0.37913612, 0.24848671, 0.1739414, -0.059445..."
8,9,"???Clearly, the Obama administration did not l...",fake,clearly obama administration leave kind game p...,"[-0.041996665, 0.0921341, -0.08574109, -0.0806..."
9,10,Retraction—Hydroxychloroquine or chloroquine w...,fake,retraction hydroxychloroquine chloroquine with...,"[-0.0713542, 0.105158806, 0.010012725, 0.06898..."


In [29]:
Val_data.head(10)

,id,tweet,label,cleanText,vec
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...,"[-0.1647581, 0.11117734, -0.101609774, -0.0438..."
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...,"[-0.056094922, 0.15008162, -0.037043996, -0.15..."
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin,"[-0.45163333, 0.12016667, -0.31364572, -0.2237..."
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...,"[-0.039578132, 0.05074875, 0.18624204, -0.0340..."
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...,"[-0.16139063, -0.030738456, -0.009159546, 0.02..."
5,6,No one can leave managed isolation for any rea...,real,one leave managed isolation reason without ret...,"[0.065014094, 0.121171474, -0.10379831, 0.0553..."
6,7,#IndiaFightsCorona India has one of the lowest...,real,indiafightscorona india one lowest covid morta...,"[-0.28247982, 0.28573245, 0.09931266, 0.001561..."
7,8,RT @WHO: #COVID19 transmission occurs primaril...,real,covid transmission occurs primarily direct ind...,"[-0.098666914, 0.026063, -0.26585433, -0.07388..."
8,9,News and media outlet ABP Majha on the basis o...,fake,news media outlet abp majha basis internal mem...,"[0.04108065, 0.14077319, 0.04265085, 0.1151359..."
9,10,???Church services can???t resume until we???r...,fake,church services resume vaccinated says bill gates,"[-0.079733714, 0.17705144, 0.03183934, -0.0623..."


In [30]:
Test_data.head(10)

,id,tweet,cleanText,vec
0,1,Our daily update is published. States reported...,daily update published states reported tests n...,"[-0.10062275, 0.15650463, 0.011188688, -0.0582..."
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid,"[-0.42387, 0.25886333, -0.18360865, -0.1030133..."
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...,"[-0.10990975, 0.09470298, -0.027744997, -0.094..."
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...,"[-0.19326457, 0.26226482, -0.047871236, -0.038..."
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...,"[-0.04292045, 0.118151665, 0.09933858, 0.17261..."
5,6,Low #vitaminD was an independent predictor of ...,low vitamind independent predictor worse progn...,"[-0.3273435, 0.29475576, -0.106898755, 0.0439,..."
6,7,A common question: why are the cumulative outc...,common question cumulative outcome numbers sma...,"[-0.15024574, 0.17372884, -0.035834294, 0.0162..."
7,8,The government should consider bringing in any...,government consider bringing new national lock...,"[-0.07501122, 0.05742323, 0.019512538, -0.1137..."
8,9,Our daily update is published. We’ve now track...,daily update published tracked million tests y...,"[-0.17294848, 0.19076625, -0.024170194, 0.0047..."
9,10,Breakdown of testing: 4 air crew 97 hotel &amp...,breakdown testing air crew hotel amp health st...,"[0.08658646, 0.1944354, 0.16256866, -0.0887498..."


**Splitting Dataset**

In [31]:
X_train = Train_data['vec'].to_numpy()
X_train = X_train.reshape(-1, 1)

X_test = Val_data['vec'].to_numpy()
X_test = X_test.reshape(-1, 1)

In [32]:
X_train.shape, X_test.shape

((1000, 1), (1000, 1))

In [33]:
X_train = np.concatenate(np.concatenate(X_train, axis = 0), axis = 0).reshape(-1, 300)
X_test = np.concatenate(np.concatenate(X_test, axis = 0), axis = 0).reshape(-1, 300)

In [34]:
X_train.shape, X_test.shape

((1000, 300), (1000, 300))

In [35]:
y_train = Train_data['label']
y_test = Val_data['label']

**Model Evaluation**

In [36]:
dict_classifiers = {
    "Nearest Neighbors":KNeighborsClassifier(3),
    "LogisticRegression":LogisticRegression(),
    #"NuSVC":NuSVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost":AdaBoostClassifier(),
    "GaussianNB":GaussianNB(),
    "LinearDiscriminant":LinearDiscriminantAnalysis(),
    "QuadraticDiscriminant":QuadraticDiscriminantAnalysis(),
    "GradientBoost":GradientBoostingClassifier(),
    #"MultinomialNB":MultinomialNB(),
    "SGD":SGDClassifier(),
    "LGBM":LGBMClassifier(),
    #"XGB":XGBClassifier(),
    "Gaussian Process Classifier":GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Support Vector Machine 1":SVC(kernel="linear", C=0.025),
    "Support Vector Machine 2":SVC(gamma=2, C=1)
}

In [37]:
def evaluation_matrix(model, testX, testy):
    # predict probabilities for test set
    yhat_probs = model.predict(testX)
    #print(yhat_probs)
    # predict crisp classes for test set
    #yhat_classes = model.predict_classes(testX)
    # reduce to 1d array
    #yhat_probs = yhat_probs[:, 0]
    #print(yhat_probs)
    #yhat_classes = yhat_classes
    yhat_classes = yhat_probs
    #accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes, average='weighted')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(testy, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    #auc = roc_auc_score(testy, yhat_probs, multi_class = 'ovr')
    #print('ROC AUC: %f' % auc)
    #confusion matrix
    matrix = confusion_matrix(testy, yhat_classes)
    print(matrix)
    return accuracy, precision, recall, f1, kappa

In [38]:
def batch_classify(dict_classifiers, X_train,X_test,y_train,y_test):
    no_classifiers = len(dict_classifiers)
    dict_super_models = {}

    for classifier_name,classifier in list(dict_classifiers.items())[:no_classifiers]:
        print(classifier_name)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        test_score = classifier.score(X_test, y_test)
        accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, X_test, y_test)
        y_pred = classifier.predict(X_test)
        print('train score:{%0.2f} test_score:{%0.2f}' %(train_score, test_score))
        dict_super_models[classifier_name] = {'Supervised': classifier_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Kappa': kappa}   

    return dict_super_models

In [39]:
dict_super_models = batch_classify(dict_classifiers, X_train,X_test,y_train,y_test)

Nearest Neighbors
Accuracy: 0.840000
Precision: 0.846343
Recall: 0.840000
F1 score: 0.838409
Cohens kappa: 0.676130
[[354 117]
 [ 43 486]]
train score:{0.93} test_score:{0.84}
LogisticRegression
Accuracy: 0.871000
Precision: 0.871825
Recall: 0.871000
F1 score: 0.870672
Cohens kappa: 0.740252
[[392  79]
 [ 50 479]]
train score:{0.91} test_score:{0.87}
Decision Tree
Accuracy: 0.716000
Precision: 0.717824
Recall: 0.716000
F1 score: 0.713527
Cohens kappa: 0.425536
[[295 176]
 [108 421]]
train score:{0.91} test_score:{0.72}
Random Forest
Accuracy: 0.752000
Precision: 0.760110
Recall: 0.752000
F1 score: 0.747846
Cohens kappa: 0.496346
[[296 175]
 [ 73 456]]
train score:{0.85} test_score:{0.75}
MLP
Accuracy: 0.871000
Precision: 0.872504
Recall: 0.871000
F1 score: 0.870542
Cohens kappa: 0.740010
[[388  83]
 [ 46 483]]
train score:{0.96} test_score:{0.87}
AdaBoost
Accuracy: 0.816000
Precision: 0.816079
Recall: 0.816000
F1 score: 0.815711
Cohens kappa: 0.629896
[[369 102]
 [ 82 447]]
train score

In [40]:
def display_dict_models(dict_models, sort_by='Accuracy'):
#def display_dict_models(dict_models, sort_by='train_score'):
    cls = [key for key in dict_models.keys()]
    accuracy = [dict_models[key]['Accuracy'] for key in cls]
    precision = [dict_models[key]['Precision'] for key in cls]
    recall = [dict_models[key]['Recall'] for key in cls]
    f1 = [dict_models[key]['F1'] for key in cls]
    kappa = [dict_models[key]['Kappa'] for key in cls]
    #training_t = [dict_models[key]['train_time'] for key in cls]
    
    #df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),6)), columns = ['classifier', 'Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'])
    # df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),3)), columns = ['classifier', 'train_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'Accuracy'] = accuracy[ii]
        df_.loc[ii, 'Precision'] = precision[ii]
        df_.loc[ii, 'Recall'] = recall[ii]
        df_.loc[ii, 'F1'] = f1[ii]
        df_.loc[ii, 'Kappa'] = kappa[ii]
       
        #df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    return df_

In [41]:
df = display_dict_models(dict_super_models)

,classifier,Accuracy,Precision,Recall,F1,Kappa
12,Gaussian Process Classifier,0.884,0.884179,0.884,0.883862,0.766783
1,LogisticRegression,0.871,0.871825,0.871,0.870672,0.740252
4,MLP,0.871,0.872504,0.871,0.870542,0.740010
11,LGBM,0.858,0.857960,0.858,0.857925,0.714776
9,GradientBoost,0.852,0.851954,0.852,0.851963,0.702863
10,SGD,0.848,0.848231,0.848,0.848061,0.695258
13,Support Vector Machine 1,0.843,0.843407,0.843,0.842672,0.684021
0,Nearest Neighbors,0.840,0.846343,0.840,0.838409,0.676130
7,LinearDiscriminant,0.823,0.825860,0.823,0.821886,0.642517
5,AdaBoost,0.816,0.816079,0.816,0.815711,0.629896


In [42]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                  classifier &  Accuracy &  Precision &  Recall &        F1 &     Kappa \\
\midrule
           Nearest Neighbors &     0.840 &   0.846343 &   0.840 &  0.838409 &  0.676130 \\
          LogisticRegression &     0.871 &   0.871825 &   0.871 &  0.870672 &  0.740252 \\
               Decision Tree &     0.716 &   0.717824 &   0.716 &  0.713527 &  0.425536 \\
               Random Forest &     0.752 &   0.760110 &   0.752 &  0.747846 &  0.496346 \\
                         MLP &     0.871 &   0.872504 &   0.871 &  0.870542 &  0.740010 \\
                    AdaBoost &     0.816 &   0.816079 &   0.816 &  0.815711 &  0.629896 \\
                  GaussianNB &     0.773 &   0.776008 &   0.773 &  0.771157 &  0.540995 \\
          LinearDiscriminant &     0.823 &   0.825860 &   0.823 &  0.821886 &  0.642517 \\
       QuadraticDiscriminant &     0.801 &   0.845726 &   0.801 &  0.796869 &  0.608628 \\
               GradientBoost &     0.852 &   0.8